In [1]:
#provisional

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

!pip uninstall -q -y tensorflow google-colab grpcio
!pip install -q tf-nightly
#!pip install -q git+https://github.com/tensorflow/examples

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
import sys
sys.path.append('C:/Users/Adrian/Documents/examples')

from tensorflow_examples.lite.model_customization.core.data_util.text_dataloader import TextClassifierDataLoader
from tensorflow_examples.lite.model_customization.core.model_export_format import ModelExportFormat
import tensorflow_examples.lite.model_customization.core.task.text_classifier as text_classifier

from textdata_extension import from_panda
TextClassifierDataLoader.from_panda = from_panda

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

##  Load Dataset

In [3]:
rev_frame = pd.read_csv("Reviews.csv")

In [4]:
# df is the copy we process for sentimental analysis
df=rev_frame.copy()
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Data cleaning and preprocessing

Regard only 'Text' and 'Score' for analysis and rename them to 'review' and 'rating'

In [5]:
df=df[['Text','Score']]
df['review']=df['Text']
df['rating']=df['Score']
df.drop(['Text','Score'],axis=1,inplace=True)

In [6]:
print(df.shape)
df.head()

(568454, 2)


,review,rating
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


Check for null values:

In [7]:
print(df['rating'].isnull().sum())
df['review'].isnull().sum()  # no null values.

0


0

Remove duplicates:

In [8]:
# remove duplicates/ for every duplicate we will keep only one row of that type. 
df.drop_duplicates(subset=['rating','review'],keep='first',inplace=True) 

In [9]:
# now check the shape. note that shape is reduced which shows that we did has duplicate rows.
size = df.shape
print(size)
df.head()

(393675, 2)


,review,rating
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


Check datatypes and prepare to convert into Tensorflow data object

In [10]:
TextClassifierDataLoader.from_panda(df,size[0])

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'> <class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


UnboundLocalError: local variable 'label_names' referenced before assignment

Next step should be to create a model *text_classifier.create(dataloader)*

In [ ]:
text_ds = tf.data.Dataset.from_tensor_slices( tf.cast(df['review'].values, tf.string) )